In [13]:
# NLTK and visualization
import nltk

import sys 
sys.path.append('..')
#from nltk2tikz import show_parse

# koristenje dokazatelja (prover)
prover = nltk.Prover9()
prover.config_prover9(r'C://Tools//Prover9-Mace4//bin-win32' )

import os
os.environ["Prover9"] = "C://Tools//Prover9-Mace4//bin-win32"

my_value = os.environ.get("Prover9")


In [27]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#!jupyter nbconvert --to html Semantika.ipynb

# Analiziranje značenja prirodnog jezika

## Motivacija: "Brute force" pristup analiziranja značenja prirodnog jezika

> *Scenarij*: Želimo da računalo može odgovoriti na pitanje: *What cities are located in China?* 

> Brute-Force: prirodni jezik -> FBG -> SQL :-)
> * Uska domena značenja: prijestolnica + država + populacija = SQL DB
> * Formirati FBG koja upit prirodnog jezika pretvara u SQL upit ...  npr. `SELECT Country FROM city_table WHERE City = 'athens'`

In [2]:
import nltk
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [3]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in Greece'
trees = list(cp.parse(query.split()))

print(trees[0])
    
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q,':-)')

(S[SEM=(SELECT, City FROM city_table, WHERE, , , Country="greece")]
  (NP[SEM=(SELECT, City FROM city_table)]
    (Det[SEM='SELECT'] What)
    (N[SEM='City FROM city_table'] cities))
  (VP[SEM=(, , Country="greece")]
    (IV[SEM=''] are)
    (AP[SEM=(, Country="greece")]
      (A[SEM=''] located)
      (PP[SEM=(, Country="greece")]
        (P[SEM=''] in)
        (NP[SEM='Country="greece"'] Greece)))))
SELECT City FROM city_table WHERE Country="greece" :-)


In [4]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: print(r[0])

LookupError: 
**********************************************************************
  Resource [93mcity_database[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('city_database')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/city_database/city.db[0m

  Searched in:
    - 'C:\\Users\\Domagoj/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2032.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2032.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2032.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\Domagoj\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


## Logika u NLTK

**CILJ:** logičkim zaključivanjem (eksplicitnim formalnim pristupom) utvrditi je li skup rečenica logički konzistentan, tj. kao istinosni sudovi ne kontradiktiraju jedne drugima (neovisno o leksičkom značenju riječi).

### Propozicijska logika u NLTK

* propozicijski simboli ($P,Q,R,\ldots$) s pridruženim valuacijama $\{\top, \bot\}$ + logički operatori ($ \neg \, \land \, \lor \, \rightarrow \, \leftrightarrow  $)
* utvrditi $(P_1,P_2\ldots,P_n) \vdash Q $ gdje su $P_i$ pretpostavke, a $Q$ zaključak (korištenjem rač. log. alata)

Mogu li sljedeće 2 rečenice biti konzistentne?

scenarij 1:
* $SnF$ = 'Sylvania is to the north of Freedonia'
* $FnS$ = 'Freedonia is to the north of Sylvania'

scenarij 2:
* $SnF$ = 'Sylvania is to the north of Freedonia'
* $nFnS$ = 'Freedonia is not to the north of Sylvania'

In [14]:
# ucitavanje simbola
read_expr = nltk.sem.Expression.fromstring
# logicke propozicije
SnF = read_expr('SnF')
FnS = read_expr('FnS')
nFnS = read_expr('-FnS')
R = read_expr('SnF -> -FnS')


# zakljucak 1:
print(prover.prove(FnS, [SnF, R]))
#zakljucak 2: 
print(prover.prove(nFnS,[SnF,R]))

False
True


## Predikatna logika (logika prvog-reda)

Obuhvaća predikatnu logiku s dodatnim mehanizmima:
* logičke konstante $c$ - sadrže uvijek iste vrijednosti 
* varijable $x_i,i=1,2,\ldots,n$ - poprimaju vrijednosti ovisno o modelu
* predikati $P(x_1,x_2,\ldots,x_n)$
* kvantifikatori $\exists, \forall$

Interpretacija rečenice kao logika prvog reda:
 * "He is a dog and he disappeared".
 * $\exists x. (dog(x)\,\wedge\,disappear(x))$ = "Postoji objekt koji je pas i on je nestao."


## SnF primjer kao dokazivanje teorema u FOL
Isti problem iskazan u predikatnoj logici:

scenarij 1: 
* argumenti $x,y$, predikat $P(x,y)$ predstavlja je li $x$ sjeverno od $y$, prema tome:
  
  $SnF := P(x,y)$
  
  $nFnS := \neg P(y,x)$
  
  $R$ := $\forall x, \forall y \, (P(x,y) \rightarrow \neg P(y,x))$: 


In [6]:
nFnS = read_expr('-north_of(f, s)')
SnF = read_expr('north_of(s, f)')
FnS = read_expr('north_of(f, s)')
# scenarij 1
R = read_expr('all x. all y. (north_of(x, y) -> -north_of(y, x))')
prover.prove(nFnS, [SnF, R])
# scanarij 2
prover.prove(FnS, [SnF, R])

False

# Istinom uvjetovana semantika

Jezik FOL-a (prema Montague gramatici): 
  1. Ako je $P$ predikat tipa $\langle e^n,t\rangle$ i $\alpha_1,\alpha_2,\ldots,\alpha_n$ izrazi tipa $e$, tada je $P(\alpha_1,\alpha_2,\ldots,\alpha_n)$ tipa $t$
  2. Ako su $\alpha$ i $\beta$ tipa $e$, tada $\alpha=\beta$ i $\alpha!=\beta$ su tipa $t$
  3. Ako je $\varphi$ tipa $t$, onda je $\neg\varphi$ tipa $t$
  4. Ako su $\varphi$ i $\psi$ tipa $t$, onda su $\varphi \wedge \psi, \varphi \vee \psi, \varphi \rightarrow \psi, \varphi \leftrightarrow \psi$ tipa $t$
  5. Ako je $\varphi$ tipa $t$ i $x$ varijabla tipa $e$, onda $\exists x.\varphi$ i $\forall x.\varphi$ su tipa $t$

Za dani FOL jezik $L$, model $M$ je par $(D,Val)$ gdje je $D$ neprazan skup (domena) i $Val$ valuacijska funkcija, vrijedi:
* $ Val(c)\in D$, $\forall c\in L$ konstanta
* $ Val(P)\colon D^n\to \{\top,\bot\}$ za broj argumenata $n$ u predikatu $P$    
 


### Primjer: model propozicijske logike

In [7]:
# valuacija
val = nltk.Valuation([('P',True),('Q',True),('R',False)])
val['P']
# domena
dom = set([])
g = nltk.Assignment(dom)
# model
m = nltk.Model(dom,val)
# evaluacija
m.evaluate('(P & Q)',g)
m.evaluate('-(P | R)',g)

False

 ### Primjer: model za primjer predikatne logike
FOL primjer **postavljanja modela** i **provjera modela**:
> Bertie je dječak. 
> Olive je djevojčica.
> Cyrill je pas.
> Olive hoda. 
> Cyrill hoda.
> Bertie vidi Olive.
> Cyrill vidi Bertie.
> Olive vidi Cyrill.


In [8]:
# model: 
dom = {'b', 'o', 'c'}

# defincija Val():
v = """
    bertie => b
    olive => o
    cyril => c
    boy => {b}
    girl => {o}
    dog => {c}
    walk => {o, c}
    see => {(b, o), (c, b), (o, c)}
    """
# bertie,olive,cyril ... konstante, b,o,c ... elementi domene
# boy,girl,dog ... unarni predikati
# walk, see  ... binarni predikati
val = nltk.Valuation.fromstring(v)
print(val)

# valuacija predikata see(o,c)?
('o','c') in val['see']

# vezanje varijabli s elementima domene
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c'),('z','b')]) # {'x':'o','y':'c','z':'b'}
print('g=',g)

# evaluacija FOL formule
m = nltk.Model(dom,val)
m.evaluate('see(olive,y)',g) # pridruzivanje g zadovoljava see(olive,y)
g.purge() # ocisti g
# scenarij 1: vidi li Bertie Olive i Bertie je dječak koji ne hoda.
m.evaluate('see(bertie,olive) & boy(bertie) & -walk(bertie)',g)
# scenarij 2: Postoji li djevojčica koja hoda?
m.evaluate('exists x.(girl(x) & walk(x))',g)
# isto kao i m.evaluate('girl(x) & walk(x)',g.add('x','o'))

# scenarij 3: Tko je sve djevojčica ili dječak? 
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('girl(x) | boy(x)')
m.satisfiers(expr, 'x', g)

# scenarij 4: Ako je djevojčica, onda ona hoda. 
expr = read_expr('girl(x) -> walk(x)')
m.satisfiers(expr, 'x', g)

# scenarij 5: Ako netko hoda, onda je ono djevojčica. 
expr = read_expr('walk(x) -> girl(x)')
m.satisfiers(expr, 'x', g)


{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('b', 'o'), ('c', 'b'), ('o', 'c')},
 'walk': {('o',), ('c',)}}
g= g[b/z][c/y][o/x]


{'b', 'o'}

**ZADATAK**: Provjerite istinitost rečenice:
> Svatko se divi nekome.

FOL interpretacija (višeznačno?):
   1. $\forall x.(person(x)\rightarrow \exists y. (person(y)\, \& \, admire(x,y)))$
   2. $\exists y.(person(y) \, \& \, \forall x. (person(x)\, \rightarrow \, admire(x,y)))$
   
ako:
> Julia se divi Bruce. Bruce se divi Bruce. Matthew se divi Elsi. Elsa se divi Matthew.


In [15]:
read_expr = nltk.sem.Expression.fromstring
# valuacija
v = """
    bruce => b
    elsa => e
    julia => j
    matthew => m
    person => {b, e, j, m}
    admire => {(j, b), (b, b), (m, e), (e, m)}
"""
val = nltk.Valuation.fromstring(v)
dom = val.domain
#print(dom)
m = nltk.Model(dom,val)
g = nltk.Assignment(dom)
# slucaj 1:
expr = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')
m.satisfiers(expr,'x',g)
# slucaj 2:
#expr = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')
# m.satisfiers(expr,'y',g)


{'b', 'e', 'j', 'm'}

## Izgradnja modela

NLTK u suradnji s `Mace4` alatom u stanju je izgraditi model na temelju danih rečenica tako da pokuša izgraditi **kontramodel** za $[s_1,s_2,\ldots,-g]$. Ako uspije, znamo da ne postoji model za $[s_1,s_2,\ldots]\vdash g$.

In [16]:

read_expr = nltk.sem.Expression.fromstring
# primjer 1: je li model konzistentan?
sent1 = read_expr('all x.(man(x) -> mortal(x))')
sent2 = read_expr('man(socrates)') 
sent3 = read_expr('mortal(socrates)')
sent4 = read_expr('-mortal(socrates)')
mb = nltk.Mace(5)
# provjeri jesu li rečenice u kontradikciji
print(mb.build_model(None, [sent1, sent2,sent3])) 
print(mb.build_model(None, [sent1, sent3, sent4]))


True
False


Primjeri rečenica:
  * $s_1$ = There is a woman that every man loves. 
  * $s_2$ = Adam is a man. 
  * $s_3$ = Eve is a woman.
  
Možemo li zaključiti `g=Adam loves Eve` ?



In [17]:
# primjer 2
sent1 = read_expr('exists y. (woman(y) & all x. (man(x) -> love(x,y)))')
sent2 = read_expr('man(adam)')
sent3 = read_expr('woman(eve)')
# zaključak
g = read_expr('love(adam,eve)') 

# izgradimo model
mc = nltk.MaceCommand(g, assumptions=[sent1,sent2,sent3])
mc.build_model() # provjera izgraditi kontramodel[sent1,sent2,sent3,-g] 
print('valuation 1:\n ',mc.valuation)


# što nedostaje?
sent4 = read_expr('all x. (man(x) -> -woman(x))')
# jedinstvena Eva
sent5 = read_expr('exists y. all x. (woman(x) -> (x=y))')


valuation 1:
  {'C1': 'b',
 'adam': 'a',
 'eve': 'a',
 'love': {('a', 'b')},
 'man': {('a',)},
 'woman': {('a',), ('b',)}}


In [18]:
mc = nltk.MaceCommand(g, assumptions=[sent1,sent2,sent3,sent4,sent5])
print(mc.build_model()) # provjera [sent1,sent2,sent3,sent4,sent5,-g] izgradnjom kontramodela
print('valuation 2:\n',mc.valuation)

False
valuation 2:
 {}


# Načelo semantičke kompozicije 

Fregeovo **načelo kompozicije**: 
  > značenje cjeline je funkcija značenja dijelova i kako su oni sintaktički spajaju
  
**FBG + semantička kategorizacija + $\lambda$-kalkulus**: Koristiti FBG i njegove parsere kako bi uz pomoć $\lambda$ izraza kategorizirali rečenice u logiku prvog reda.

**Primjer**: "Cyril barks."

![](FBG_sem.png)

```
S[SEM=<?vp(?np)>] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=?v] -> IV[SEM=?v]
NP[SEM=<cyril>] -> 'Cyril'
IV[SEM=<\x.bark(x)>] -> 'barks'
```



In [19]:
%%writefile sem.fcfg
% start S
# Grammar Rules
S[SEM=<?vp(?np)>] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=?v] -> IV[SEM=?v]
# Lexical Rules
NP[SEM=<cyril>] -> 'Cyril'
IV[SEM=<\x.bark(x)>] -> 'barks'

Overwriting sem.fcfg


In [20]:
tokens = 'Cyril barks'.split()
cp = nltk.load_parser('sem.fcfg')
trees = cp.parse_all(tokens)


for tree in trees:
    print(tree.label()['SEM'])
    

tree = list(trees)[0]

print(tree.pformat_latex_qtree())
#show_parse(tree,width=500)

bark(cyril)
\Tree [.S[SEM=<bark(cyril)>]
        [.NP[SEM=<cyril>] Cyril ]
        [.VP[SEM=<\x.bark(x)>] [.IV[SEM=<\x.bark(x)>] barks ] ] ]


## $\lambda$-kalkulus
* **$\lambda$-kalkulus** = formalni sustav u matematičkoj logici koje izražava *računanje* preko apstrakcije funkcije i primjene iste korištenjem vezivanje varijable i zamjenama.
* $x$ ... varijabla
* $\lambda x.M$ ... funkcijska apstrakcija
* $M ~ N$ ... primjena funkcije na argument
* redukcije: 
    > $\beta$-redukcija: $((\lambda x.M)(\beta)) \to (M[\beta/x])]$
    
    > $\alpha$-konverzija: $(\lambda x.M[x]) \to (\lambda y.M[y])$

Primjer: 
* FO-logika: $(walk(x) \, \& \,dog(x))$ 
* $\lambda$-kalkulus: $\lambda x.(walk(x) \, \& \, dog(x))$ <br />
  interpretacija: "biti $x$ takav da $x$ hoda i $x$ je pas."



In [21]:
# NLTK primjeri za lambda kalkulus
read_expr = nltk.sem.Expression.fromstring
expr = read_expr(r'\x.(walk(x) & chew_gum(x))')
print(read_expr(r'\x.(walk(x) & chew_gum(x))'))

# beta-redukcija: zamjena vezane varijable s argumentom
expr = read_expr(r'\x.(walk(x) & chew_gum(x))(gerald)')
print(expr.simplify()) # izvedba redukcije
# vezivanje 2 varijable 
print(read_expr(r'\x.\y.(dog(x) & own(y, x))(cyril)').simplify())
print(read_expr(r'\x y.(dog(x) & own(y, x))(cyril, angus)').simplify())
# alpha-konverzija
expr = read_expr('exists x.P(x)')
_expr = expr.alpha_convert(nltk.sem.Variable('z'))
print(expr==_expr)
print(_expr)
# Funkcijske apstrakcije kao argumenti
expr = read_expr(r'\P.(exists x.P(x))(\y.see(y,x))')
print(expr.simplify())

\x.(walk(x) & chew_gum(x))
(walk(gerald) & chew_gum(gerald))
\y.(dog(cyril) & own(y,cyril))
(dog(cyril) & own(angus,cyril))
True
exists z.P(z)
exists z1.see(z1,x)


## Leksičko-semantički izazovi za $\lambda$-kalkulus



### Kvantificirani NP
**Primjer**:
1. NL: "A dog barks" 
2. FOL:$\exists x.(dog(x)\, \& \, bark(x))$
3. $\lambda$-kalkulus?


Želimo `S[SEM=<exists x.(dog(x) & bark(x))>]` na korijenu FBG stabla!
* $\lambda$-izraz: `\P.exists x.(dog(x) & P(x))`
* $\beta$-redukcija: `\P.exists x.(dog(x) & P(x))(\x.bark(x))`

dakle `S[SEM=<?np(\x.bark(x))>]` daje željeni rezultat!


In [22]:
read_expr = nltk.sem.Expression.fromstring
expr1 = read_expr(r'\Q P.exists x.(Q(x) & P(x))')
expr2 = read_expr(r'\x.dog(x)')
expr3 = read_expr(r'\x.bark(x)')
np = nltk.sem.ApplicationExpression(expr1, expr2)
print(np.simplify())
sem = nltk.sem.ApplicationExpression(np, expr3)
print(sem.simplify())

\P.exists x.(dog(x) & P(x))
exists x.(dog(x) & bark(x))


### Tranzitivni glagoli
**Primjer**:
1. NL: "Angus chases a dog." 
2. FOL:$\exists x.(dog(x)\, \& \, chases(angus,x))$
3. $\lambda$-kalkulus?

želimo $\lambda$-izraz: `\y.exists x.(dog(x) & chase(y, x))`

Kako parametrizirati ostale predikate u izrazu?


In [23]:
read_expr = nltk.sem.Expression.fromstring


tvp = read_expr(r'\X x.X(\y.chase(x,y))')
np = read_expr(r'(\P.exists x.(dog(x) & P(x)))')

vp = nltk.sem.ApplicationExpression(tvp, np)
print(vp)
print(vp.simplify())
PropN = read_expr(r'\P.P(angus)') # interpretacija imena kao funkcije
print(vp.simplify())
sem = nltk.sem.ApplicationExpression(PropN,vp)
print(sem)
print(sem.simplify())


(\X x.X(\y.chase(x,y)))(\P.exists x.(dog(x) & P(x)))
\x.exists z2.(dog(z2) & chase(x,z2))
\x.exists z3.(dog(z3) & chase(x,z3))
\P.P(angus)((\X x.X(\y.chase(x,y)))(\P.exists x.(dog(x) & P(x))))
exists z4.(dog(z4) & chase(angus,z4))


In [51]:
# ispisi FBG sa obilježjima (semantička kategorizacija)
nltk.data.show_cfg('grammars/book_grammars/simple-sem.fcfg')

% start S
# Grammar Rules
S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]
NP[NUM=?n,SEM=<?det(?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
NP[LOC=?l,NUM=?n,SEM=?np] -> PropN[LOC=?l,NUM=?n,SEM=?np]
Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]
VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?v]
VP[NUM=?n,SEM=<?v(?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=<?v(?obj,?pp)>] -> DTV[NUM=?n,SEM=?v] NP[SEM=?obj] PP[+TO,SEM=?pp]
PP[+TO, SEM=?np] -> P[+TO] NP[SEM=?np]
# Lexical Rules
PropN[-LOC,NUM=sg,SEM=<\P.P(angus)>] -> 'Angus'
PropN[-LOC,NUM=sg,SEM=<\P.P(cyril)>] -> 'Cyril'
PropN[-LOC,NUM=sg,SEM=<\P.P(irene)>] -> 'Irene'
Det[NUM=sg,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'every'
Det[NUM=pl,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'all'
Det[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'some'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'a'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'an'
N[NUM=sg,SEM=<\x.man(x)>] -> 'man'
N[NUM=sg,SEM=<\x.girl(x)>] -> 'girl'
N[NUM=sg,SEM=<\x.b

Iskoristit ćemo `simple-sem.fcfg` kako bi ispitali istinost tvrdnji i ujedno ponuditi mogućnost vidjeti produkciju semantičkih obilježja i valuacije.

In [24]:
from nltk import load_parser
# FCFG grafika
parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)
# test rečenica
sentence = 'Angus gives a bone to every dog'
# tokenizacija
tokens = sentence.split()
# obilježje semantike nakon produkcije FBG gramatike
for tree in parser.parse(tokens):
    print(tree.label()['SEM'])

all z6.(dog(z6) -> exists z5.(bone(z5) & give(angus,z5,z6)))


In [25]:
sents = ['Irene walks', 'Cyril bites an ankle']
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
# primijeni FBG na dane rečenice i ispiši semnatičko obilježje
for results in nltk.interpret_sents(sents, grammar_file):
        for (synrep, semrep) in results:
            print(semrep)

walk(irene)
exists z7.(ankle(z7) & bite(cyril,z7))


In [26]:
# valuacija modela 
# postavljamo model
v = """
    bertie => b
    olive => o
    cyril => c
    boy => {b}
    girl => {o}
    dog => {c}
    walk => {o, c}
    see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)

g = nltk.Assignment(val.domain)
m = nltk.Model(val.domain, val)
sent = 'Cyril sees every boy'
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
results = nltk.evaluate_sents([sent], grammar_file, m, g)[0]
# ispis semantičke reprezentacije i valuacije iste
for (synrep, semrep, value) in results:
    #print(synrep)
    print(semrep)
    print(value)

all z8.(boy(z8) -> see(cyril,z8))
True


# Zaključak

 * FOL logika u prezentiranju značenja prirodnog jezika, dokazivači teorema kao alati za rezoniranje o značenju
 * kompozitnost značenja ostvarena korištenjem FOL + $\lambda$-kalkulus
 * redukcije u korištenju definiranja argumenata
 * konstrukcija modela temeljena na valuaciji: interpretacija nelogičkih konstanti
 * SEM kao kategorija u FBG stvara slože semantičke izraze